# PyBullet

This is a series of notebooks related to PyBullet, created after following the official guide.

Overview

1. Introduction
    1.1 Hello World
        - Client-server architecture and connection mode
        - Most important functions
        - URDF files
        - GUI control
    1.2 Shapes: Visual & Collision
    1.3 Simulation: `stepSimulation()`, `performCollisionDetection()`
    1.4 Pose Transformations
2. Controlling Robots

## 2. Controlling Robots

A robot described with a URDF has
- a base, which can be fully fixed (0 DoF) or free (6 DoF),
- and, optionally, links, connected by joints

Links and joints have have indices [0-(n-1)] and frames, expressed wrt. the parent links. By convention, base link index is -1.

<img src="pics/robot_link_joint_convention.png">

In [5]:
import pybullet as p
import time
import pybullet_data

In [6]:
pybullet_data.getDataPath()

'C:\\Users\\msagardia\\AppData\\Roaming\\Python\\Python37\\site-packages\\pybullet_data'

In [14]:
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
startPosOrn = p.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (100): # 10000
    p.stepSimulation() # optional: physicsClientId
    time.sleep(1./240.)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)

(1.5117627740528593e-08, 1.4889593646656074e-06, 0.47099230897457317) (-3.379035448734749e-08, 3.997113849372463e-09, 9.265643796039036e-08, 0.9999999999999951)


In [12]:
#p.disconnect()

In [15]:
# Get number of joints
p.getNumJoints(boxId)

15

In [21]:
# Get joint information: returned list:
# jointIndex
# jointName
# jointType
# qIndex
# uIndex
# flags
# Some values defined in the URDF:
# damping, friction, lower limit, upper limit, max force, max velocity,
# link name, joint axis, parent frame pos, parent frame ordientation, parent index
p.getJointInfo(boxId,14)

(14,
 b'tobox',
 4,
 -1,
 -1,
 0,
 0.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 b'box',
 (0.0, 0.0, 0.0),
 (0.0, 0.1214, 0.1214),
 (0.0, 0.0, 0.0, 1.0),
 13)

### 2.1 Joint Control

We select a control mode; each mode has its parameters:
- `POSITION_CONTROL`: `positionGain`, `maxVelocity`, `velocityGain`: constraint error minimized = positionGain * (desired_position-actual_position) + velocityGain * (desired_velocity-actual_velocity)
- `VELOCITY_CONTROL`: `velocityGain`: desired_velocity-actual_velocity
- `TORQUE_CONTROL`: torque applied instantly, effective only when calling `stepSimulation()`; generally it is better to use the prior two first
- `PD_CONTROL`

Example call with `setJointMotorControl2()`:
```python
maxForce = 500
mode = p.VELOCITY_CONTROL
p.setJointMotorControl2(bodyUniqueId=objUid, # returned by loadURDF
                        jointIndex=0, # jointIndex = linkIndex
                        controlMode=mode,
                        targetVelocity = targetVel,
                        force = maxForce)
```



Notes:
- Use `setJointMotorControl2()` or `setJointMotorControlArray()`, not `setJointMotorControl()`.
- `setJointMotorControlArray()` takes the same parameters as `setJointMotorControl2()` but in an array of `ints`.
- `setJointMotorControlMultiDof()` and `setJointMotorControlMultiDofArray()` support spherical joints.

### 2.2 Joint States

In [23]:
# Get joint state:
# postion (1)
# velocity (1)
# reaction forces (fx, fy, fz, tx. ty. tz) = 0 if no sensor
# appliedJointMotorTorque (1): applied during setpSimulation()
p.getJointState(boxId,14)

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)